In [1]:
import pandas as pd
import numpy as np
import extracted_data_reader

In [2]:
X = extracted_data_reader.read_data_ex_2(1)


Shortest video: 032 GT.1.2.1.csv	53


In [3]:
from tsai.all import *

C:\Users\Gabriele\AppData\Roaming\Python\Python310\site-packages\tsai\imports.py:346: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem . (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:84.)
  return torch.device("cpu")


In [6]:
Y = [1 for _ in range(len(X))]
splits = [[i for i in range(len(X) //2)], [i for i in range(len(X) // 2, len(X))]]

tfms  = [None, [Categorize()]]
dsets = TSDatasets(X, Y, tfms=tfms, splits=splits, inplace=True)
dsets

(#285) [(TSTensor([1.0842021724855044e-19], device=cpu, dtype=torch.float32), TensorCategory(0)), (TSTensor([1.7438002824783325], device=cpu, dtype=torch.float32), TensorCategory(0)), (TSTensor([1.0842021078620191e-19], device=cpu, dtype=torch.float32), TensorCategory(0)), (TSTensor([1.7445579767227173], device=cpu, dtype=torch.float32), TensorCategory(0)), (TSTensor([2.0], device=cpu, dtype=torch.float32), TensorCategory(0)), (TSTensor([1.7445838451385498], device=cpu, dtype=torch.float32), TensorCategory(0)), (TSTensor([1.9999998807907104], device=cpu, dtype=torch.float32), TensorCategory(0)), (TSTensor([1.744422435760498], device=cpu, dtype=torch.float32), TensorCategory(0)), (TSTensor([-1.0842021724855044e-19], device=cpu, dtype=torch.float32), TensorCategory(0)), (TSTensor([1.7441548109054565], device=cpu, dtype=torch.float32), TensorCategory(0))] ...]